## Генерация рекомендаций с использованием LLM: генеративный подход

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:
students = pd.read_json("hse_students_combined_50.json").iloc[:20]
projects = pd.read_json("hse_all_projects.json").iloc[:20]
matches = pd.read_json("student_project_manual_matching_top20.json")

In [4]:
def format_student(row):
    interests = ", ".join(row["Научные интересы (Фолксономия)"])
    return f"Интересы: {interests}. О себе: {row['Рассказ о себе']}"

students["text"] = students.apply(format_student, axis=1)

# Текстовое представление проектов
def format_project(row):
    return f"{row['Название проекта']}. {row['Описание проекта']} Сроки: {row['Сроки исполнения']}"

projects["text"] = projects.apply(format_project, axis=1)

In [5]:
students["GPA_norm"] = (students["GPA"] - 6) / 4

In [6]:
def precision_at_3(similarity_matrix, students, projects, matches):
    precisions = []
    for i, fio in enumerate(students["ФИО"]):
        top_3_indices = similarity_matrix[i].argsort()[-3:][::-1]
        top_3_projects = [projects[j] for j in top_3_indices]
        gt = matches[matches["ФИО"] == fio][["1-й проект", "2-й проект", "3-й проект"]].values.flatten().tolist()
        num_relevant = sum(1 for p in top_3_projects if p in gt)
        precisions.append(num_relevant / 3)
    return np.mean(precisions)

In [7]:
!pip install langchain chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 2.7 MB/s eta 0:

In [8]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00


In [9]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import HumanMessage, SystemMessage
from langchain.docstore.document import Document
from langchain.vectorstores.utils import filter_complex_metadata

In [10]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2025-06-08 04:35:47--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11316 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.05K  --.-KB/s    in 0.001s  

2025-06-08 04:35:47 (16.3 MB/s) - ‘utils.py’ saved [11316/11316]



In [11]:
!pip install langchain langchain-openai openai langchainhub google-search-results faiss-cpu langchain-experimental langserve -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 940.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 21.1 MB/s eta 0:00:00


In [12]:
from utils import ChatOpenAI

course_api_key = '...'

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [13]:
def format_project(row):
    return (f"Название проекта: {row['Название проекта']}\n"
            f"Описание: {row['Описание проекта']}\n"
            f"Сроки: {row['Сроки исполнения']}")

docs = [
    Document(page_content=format_project(row), metadata={"Название": row["Название проекта"]})
    for _, row in projects.iterrows()
]

In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = Chroma.from_documents(docs, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [16]:
def format_student_query(row):
    interests = ", ".join(row["Научные интересы (Фолксономия)"])
    return f"Интересы: {interests}\nGPA: {row['GPA']}\nО себе: {row['Рассказ о себе']}"

results = []

### Zero shot

In [17]:
for _, row in students.iterrows():
    query_text = format_student_query(row)
    retrieved_docs = retriever.get_relevant_documents(query_text)

    project_block = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = (
        f"Вот профиль студента:\n{query_text}\n\n"
        f"Из списка ниже выбери один наиболее подходящий проект. "
        f"Ответ должен содержать ТОЛЬКО НАЗВАНИЕ проекта.\n\n"
        f"{project_block}"
    )

    try:
        messages = [
            SystemMessage(content="Ты выступаешь как рекомендательная система."),
            HumanMessage(content=prompt)
        ]
        response = llm(messages).content.strip()
    except Exception as e:
        response = f"[Ошибка]: {e}"

    results.append({
        "ФИО": row["ФИО"],
        "Выбранный проект": response
    })

<ipython-input-17-06819ad460c5>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query_text)
<ipython-input-17-06819ad460c5>:18: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages).content.strip()


In [21]:
df_results = pd.DataFrame(results)
merged = df_results.merge(matches, on="ФИО")

def is_in_top3(row):
    return sum(1 for p in [row["1-й проект"], row["2-й проект"], row["3-й проект"]] if row["Выбранный проект"] in [row["1-й проект"], row["2-й проект"], row["3-й проект"]]) / 3

In [58]:
merged["Precision@3"] = merged.apply(is_in_top3, axis=1)
precision = merged["Precision@3"].mean()

print(f"\n Precision@3 (GPT + Retrieval): {precision:.2f}\n")


 Precision@3 (GPT + Retrieval): 0.55



### Chain of thoughts

In [23]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [53]:
results = []

for _, row in students.iterrows():
    query_text = format_student_query(row)
    retrieved_docs = retriever.get_relevant_documents(query_text)

    project_block = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = (
            f"Студент: {query_text}\n\n"
            f"Подумай шаг за шагом, какие проекты из списка подойдут. "
            f"В конце выведи название трех наиболее подходящих проектов.\n\n"
            f"Список проектов:\n{project_block}"
        )

    try:
        messages = [
            SystemMessage(content="Ты выступаешь как рекомендательная система."),
            HumanMessage(content=prompt)
        ]
        response = llm(messages).content.strip()
    except Exception as e:
        response = f"[Ошибка]: {e}"

    results.append({
        "ФИО": row["ФИО"],
        "Выбранный проект": response
    })

In [57]:
results[0]['Выбранный проект']

'Исходя из ваших интересов и целей, давайте проанализируем каждый проект по следующим критериям: соответствие интересам (прикладная математика, машинное обучение, обработка естественного языка) и возможность применения навыков.\n\n1. **Разработка рекомендательной системы для онлайн-курсов**\n   - Соответствие интересам: высокое (машинное обучение, обработка данных).\n   - Применение навыков: Python, SQL, Scikit-learn, Pandas.\n   - Ожидаемый результат: модуль рекомендаций, что может быть полезно для вашей цели в разработке интеллектуальных систем.\n\n2. **Разработка системы карьерного ориентирования на основе ИИ**\n   - Соответствие интересам: среднее (машинное обучение, но больше фокус на карьерном ориентировании).\n   - Применение навыков: Python, машинное обучение.\n   - Ожидаемый результат: система карьерных рекомендаций, что может быть менее связано с вашими интересами в области обучения.\n\n3. **Обработка медицинских изображений с помощью CNN**\n   - Соответствие интересам: низко